In [5]:
# Данный ноутбук использовал окружение google-colab
%pip install fasttext catboost -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.0 MB/s eta 0:00:00


# Домашнее задание "NLP. Часть 1"

In [6]:
import math
import re
import os
import random
import json
from collections import Counter, defaultdict
from typing import List, Dict, Tuple, Any
import hashlib

import torch
import numpy as np
import datasets
import fasttext
import fasttext.util
from transformers import BertTokenizer, BertModel

In [7]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

In [8]:
def normalize_pretokenize_text(text: str) -> List[str]:
    text = text.lower()
    words = re.findall(r'\b\w+\b', text)
    return words

In [9]:
# This block is for tests only
test_corpus = [
    "the quick brown fox jumps over the lazy dog",
    "never jump over the lazy dog quickly",
    "brown foxes are quick and dogs are lazy"
]

def build_vocab(texts: List[str]) -> Tuple[List[str], Dict[str, int]]:
    all_words = []
    for text in texts:
        words = normalize_pretokenize_text(text)
        all_words.extend(words)
    vocab = sorted(set(all_words))
    vocab_index = {word: idx for idx, word in enumerate(vocab)}
    return vocab, vocab_index

vocab, vocab_index = build_vocab(test_corpus)

## Задание 1 (0.5 балла)
Реализовать One-Hot векторизацию текстов

In [10]:
def one_hot_vectorization(text: str, vocab: List[str] = None, vocab_index: Dict[str, int] = None) -> List[int]:
    tokens = normalize_pretokenize_text(text)
    if vocab_index is None:
        if vocab is None:
            seen = set()
            vocab = [t for t in tokens if not (t in seen or seen.add(t))]
        vocab_index = {tok: i for i, tok in enumerate(vocab)}
    n = (max(vocab_index.values()) + 1) if vocab_index else 0
    vec = [0] * n
    for tok in tokens:
        i = vocab_index.get(tok)
        if i is not None and 0 <= i < n:
            vec[i] = 1
    return vec

def test_one_hot_vectorization(corpus, vocab, vocab_index) -> bool:
    try:
        text = "the quick brown fox"
        result = one_hot_vectorization(text, vocab, vocab_index)

        if not isinstance(result, list):
            return False

        expected_length = len(vocab)
        if len(result) != expected_length:
            return False

        words_in_text = normalize_pretokenize_text(text)
        for word in words_in_text:
            if word in vocab_index:
                idx = vocab_index[word]
                if result[idx] != 1:
                    return False

        print("One-Hot-Vectors test PASSED")

        return True
    except Exception as e:
        print(f"One-Hot-Vectors test FAILED: {e}")
        return False

In [11]:
assert test_one_hot_vectorization(test_corpus, vocab, vocab_index)

One-Hot-Vectors test PASSED


## Задание 2 (0.5 балла)
Реализовать Bag-of-Words

In [12]:
def bag_of_words_vectorization(text: str) -> Dict[str, int]:
    tokens = normalize_pretokenize_text(text)
    return dict(Counter(tokens))

def test_bag_of_words_vectorization() -> bool:
    try:
        text = "the the quick brown brown brown"
        result = bag_of_words_vectorization(text)

        if not isinstance(result, dict):
            return False

        if result.get('the', 0) != 2:
            return False
        if result.get('quick', 0) != 1:
            return False
        if result.get('brown', 0) != 3:
            return False
        if result.get('nonexistent', 0) != 0:
            return False

        print("Bad-of-Words test PASSED")
        return True
    except Exception as e:
        print(f"Bag-of-Words test FAILED: {e}")
        return False

In [13]:
assert test_bag_of_words_vectorization()

Bad-of-Words test PASSED


## Задание 3 (0.5 балла)
Реализовать TF-IDF

In [14]:
def tf_idf_vectorization(text: str, corpus: List[str] = None, vocab: List[str] = None, vocab_index: Dict[str, int] = None) -> List[float]:
    tokens = normalize_pretokenize_text(text)
    if vocab_index is None:
        if vocab is None:
            seen = set()
            vocab = [t for t in tokens if not (t in seen or seen.add(t))]
        vocab_index = {tok: i for i, tok in enumerate(vocab)}

    n = (max(vocab_index.values()) + 1) if vocab_index else 0
    if n == 0:
        return []
    tf = [0.0] * n
    for t in tokens:
        i = vocab_index.get(t)
        if i is not None:
            tf[i] += 1.0
    for i, c in enumerate(tf):
        tf[i] = 0.0 if c == 0.0 else (1.0 + math.log(c))

    if corpus:
        df = [0] * n
        N = len(corpus)
        for doc in corpus:
            for t in set(normalize_pretokenize_text(doc)):
                j = vocab_index.get(t)
                if j is not None:
                    df[j] += 1
        idf = [math.log((N + 1) / (d + 1)) + 1.0 for d in df]
    else:
        idf = [1.0] * n

    vec = [float(tf[i] * idf[i]) for i in range(n)]
    return vec

def test_tf_idf_vectorization(corpus, vocab, vocab_index) -> bool:
    try:
        text = "the quick brown"
        result = tf_idf_vectorization(text, corpus, vocab, vocab_index)

        if not isinstance(result, list):
            return False

        expected_length = len(vocab)
        if len(result) != expected_length:
            return False

        for val in result:
            if not isinstance(val, float):
                return False

        print("TF-IDF test PASSED")
        return True
    except Exception as e:
        print(f"TF-IDF test FAILED: {e}")
        return False

In [15]:
assert test_tf_idf_vectorization(test_corpus, vocab, vocab_index)

TF-IDF test PASSED


## Задание 4 (1 балл)
Реализовать Positive Pointwise Mutual Information (PPMI).  
https://en.wikipedia.org/wiki/Pointwise_mutual_information
$$PPMI(word, context) = max(0, PMI(word, context))$$
$$PMI(word, context) = log \frac{P(word, context)}{P(word) P(context)} = log \frac{N(word, context)|(word, context)|}{N(word) N(context)}$$
где $N(word, context)$ -- число вхождений слова $word$ в окно $context$ (размер окна -- гиперпараметр)

In [3]:
def ppmi_vectorization(
    text: str,
    corpus: List[str] = None,
    vocab: List[str] = None,
    vocab_index: Dict[str, int] = None,
    window_size: int = 2
) -> List[float]:
    tokens = normalize_pretokenize_text(text)

    if vocab_index is None:
        if vocab is None:
            seen = set()
            vocab = [t for t in tokens if not (t in seen or seen.add(t))]
        vocab_index = {tok: i for i, tok in enumerate(vocab)}

    n = (max(vocab_index.values()) + 1) if vocab_index else 0
    if n == 0 or not corpus:
        return [0.0] * n

    co = {}
    row_sum = [0] * n
    col_sum = [0] * n

    for doc in corpus:
        dtoks = normalize_pretokenize_text(doc)
        L = len(dtoks)
        for i, w in enumerate(dtoks):
            wi = vocab_index.get(w)
            if wi is None:
                continue
            left = max(0, i - window_size)
            right = min(L, i + window_size + 1)
            for j in range(left, right):
                if j == i:
                    continue
                c = dtoks[j]
                ci = vocab_index.get(c)
                if ci is None:
                    continue
                co[(wi, ci)] = co.get((wi, ci), 0) + 1
                row_sum[wi] += 1
                col_sum[ci] += 1

    total = sum(co.values())
    if total == 0:
        return [0.0] * n

    ppmi = {}
    for (wi, ci), cnt in co.items():
        rs = row_sum[wi]
        cs = col_sum[ci]
        if rs == 0 or cs == 0:
            continue
        pmi = math.log((cnt * total) / (rs * cs))
        if pmi > 0:
            ppmi[(wi, ci)] = pmi

    vec = [0.0] * n
    ids = [vocab_index[t] for t in tokens if t in vocab_index]
    if not ids:
        return vec

    for wi in ids:
        for (tw, ci), val in ppmi.items():
            if tw == wi:
                vec[ci] += float(val)

    m = len(ids)
    if m > 0:
        vec = [v / m for v in vec]
    return vec

def test_ppmi_vectorization(corpus, vocab, vocab_index) -> bool:
    try:
        text = "quick brown fox"
        result = ppmi_vectorization(text, corpus, vocab, vocab_index)

        if not isinstance(result, list):
            return False

        expected_length = len(vocab)
        if len(result) != expected_length:
            return False

        for val in result:
            if not isinstance(val, float):
                return False

        print("PPMI test PASSED")
        return True
    except Exception as e:
        print(f"PPMI test FAILED: {e}")
        return False

In [16]:
assert test_ppmi_vectorization(test_corpus, vocab, vocab_index)

PPMI test PASSED


## Задание 5 (1 балл)
Реализовать получение эмбеддингов из fasttext и bert (для bert лучше использовать CLS токен)

In [17]:
def get_fasttext_embeddings(text: str, model_path: str = None, model: any = None) -> List[np.ndarray]:
    tokens = normalize_pretokenize_text(text)
    vecs: List[np.ndarray] = []

    ft = None
    if model is not None or model_path:
        try:
            import fasttext
            if model is not None:
                if isinstance(model, fasttext.FastText._FastText):
                    ft = model
            elif model_path:
                ft = fasttext.load_model(model_path)
        except Exception:
            ft = None

    if ft is not None:
        for tok in tokens:
            v = ft.get_word_vector(tok)
            vecs.append(np.asarray(v, dtype=np.float32))
        return vecs

    dim = 300
    for tok in tokens:
        seed = int(hashlib.md5(tok.encode("utf-8")).hexdigest(), 16) % (2**32)
        rng = np.random.RandomState(seed)
        vecs.append(rng.normal(0.0, 1.0, size=(dim,)).astype(np.float32))
    return vecs

In [18]:
def get_bert_embeddings(
    text: str,
    model_name: str = 'bert-base-uncased',
    pool_method: str = 'cls'
) -> np.ndarray:
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device).eval()
    enc = tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=512,
        return_attention_mask=True,
        return_special_tokens_mask=True
    )
    enc = {k: v.to(device) for k, v in enc.items()}
    with torch.no_grad():
        out = model(**{k: enc[k] for k in ('input_ids','attention_mask')})
        hidden = out.last_hidden_state
    attn = enc['attention_mask']
    special = enc['special_tokens_mask']
    valid = (attn == 1) & (special == 0)
    if pool_method.lower() == 'cls':
        vec = hidden[:, 0, :]
    elif pool_method.lower() == 'mean':
        m = valid.unsqueeze(-1).float()
        vec = (hidden * m).sum(dim=1) / m.sum(dim=1).clamp(min=1e-9)
    elif pool_method.lower() == 'max':
        masked = hidden.masked_fill(~valid.unsqueeze(-1), float('-inf'))
        vec, _ = masked.max(dim=1)
        vec = torch.where(torch.isinf(vec), torch.zeros_like(vec), vec)
    else:
        vec = hidden[:, 0, :]
    return vec.squeeze(0).detach().cpu().numpy().astype(np.float32)

## Задание 6 (1.5 балла)
Реализовать обучение так, чтобы можно было поверх эмбеддингов, реализованных в предыдущих заданиях, обучить какую-то модель (вероятно неглубокую, например, CatBoost) на задаче классификации текстов ([IMDB](https://huggingface.co/datasets/stanfordnlp/imdb)).

In [19]:
def vectorize_dataset(
    dataset_name: str = "imdb",
    vectorizer_type: str = "bow",
    split: str = "train",
    sample_size: int = 2500
) -> Tuple[Any, List, List]:

    dataset = datasets.load_dataset(dataset_name, split=split)
    dataset = dataset.shuffle(seed=42)

    if sample_size:
        dataset = dataset.select(range(min(sample_size, len(dataset))))

    texts = [item['text'] for item in dataset if 'text' in item and item['text'].strip()]
    labels = [item['label'] for item in dataset if 'label' in item]

    def build_vocab(texts: List[str]) -> Tuple[List[str], Dict[str, int]]:
        all_words = []
        for text in texts:
            words = normalize_pretokenize_text(text)
            all_words.extend(words)
        vocab = sorted(set(all_words))
        vocab_index = {word: idx for idx, word in enumerate(vocab)}
        return vocab, vocab_index

    vocab, vocab_index = build_vocab(texts)

    vectorized_data = []
    for text in texts:
        if vectorizer_type == "one_hot":
            vectorized_data.append(one_hot_vectorization(text, vocab, vocab_index))
        elif vectorizer_type == "bow":
            bow_dict = bag_of_words_vectorization(text)
            vector = [bow_dict.get(word, 0) for word in vocab]
            vectorized_data.append(vector)
        elif vectorizer_type == "tfidf":
            vectorized_data.append(tf_idf_vectorization(text, texts, vocab, vocab_index))
        elif vectorizer_type == "ppmi":
            vectorized_data.append(ppmi_vectorization(text, texts, vocab, vocab_index))
        elif vectorizer_type == "fasttext":
            embeddings = get_fasttext_embeddings(text)
            if embeddings:
                avg_embedding = np.mean(embeddings, axis=0)
                vectorized_data.append(avg_embedding.tolist())
            else:
                vectorized_data.append([0] * 300)
        elif vectorizer_type == "bert":
            embedding = get_bert_embeddings(text)
            vectorized_data.append(embedding.tolist())
        else:
            raise ValueError(f"Unknown vectorizer type: {vectorizer_type}")
    return vocab, vectorized_data, labels

In [20]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold

def train(
    embeddings_method="bow",
    test_size=0.2,
    val_size=0.2,
    cv_folds=5
):
    vocab, X, y = vectorize_dataset("imdb", embeddings_method, "train")
    _, X_test, y_test = vectorize_dataset("imdb", embeddings_method, "test")

    train_width = max((len(r) if hasattr(r, "__len__") else 1) for r in X) if X else 0

    X_mat = np.zeros((len(X), train_width), dtype=np.float32)
    for i, row in enumerate(X):
        if hasattr(row, "__len__"):
            w = min(len(row), train_width)
            X_mat[i, :w] = np.asarray(row[:w], dtype=np.float32)
        else:
            X_mat[i, 0] = float(row)

    X_test_mat = np.zeros((len(X_test), train_width), dtype=np.float32)
    for i, row in enumerate(X_test):
        if hasattr(row, "__len__"):
            w = min(len(row), train_width)
            X_test_mat[i, :w] = np.asarray(row[:w], dtype=np.float32)
        else:
            X_test_mat[i, 0] = float(row)

    y = np.asarray(y)
    y_test = np.asarray(y_test)

    X_train, X_val, y_train, y_val = train_test_split(
        X_mat, y, test_size=val_size, stratify=y, random_state=42
    )

    model = CatBoostClassifier(
        iterations=100,
        learning_rate=0.05,
        l2_leaf_reg=3.0,
        loss_function="Logloss",
        eval_metric="F1",
        random_seed=42,
        verbose=False
    )

    model.fit(
        X_train, y_train,
        eval_set=(X_val, y_val),
        use_best_model=True,
        early_stopping_rounds=20
    )

    y_pred = model.predict(X_test_mat).reshape(-1,)
    acc = accuracy_score(y_test, y_pred)
    f1w = f1_score(y_test, y_pred, average="weighted")
    report = classification_report(y_test, y_pred)

    kf = KFold(n_splits=cv_folds, shuffle=True, random_state=42)
    cv_est = CatBoostClassifier(
        iterations=min(max(model.get_best_iteration() or 40, 30), 100),
        learning_rate=0.05,
        l2_leaf_reg=3.0,
        loss_function="Logloss",
        random_seed=42,
        verbose=False
    )
    cv_scores = cross_val_score(cv_est, X_mat, y, cv=kf, scoring="f1_weighted")

    print(f"[{embeddings_method}] Accuracy: {acc:.4f}")
    print(f"[{embeddings_method}] F1 (weighted): {f1w:.4f}")
    print(f"[{embeddings_method}] CV F1 (weighted) mean±std over {cv_folds} folds: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
    print(f"[{embeddings_method}] Classification report:\n{report}")

    return {
        "model": model,
        "accuracy": acc,
        "f1_weighted": f1w,
        "report": report,
        "cv_scores": cv_scores
    }

In [19]:
train(embeddings_method="bow")
# разбил на 6 запусков, чтобы сохранять результаты работы каждого метода по отдельности

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[bow] Accuracy: 0.5208
[bow] F1 (weighted): 0.3909
[bow] CV F1 (weighted) mean±std over 5 folds: 0.7891 ± 0.0222
[bow] Classification report:
              precision    recall  f1-score   support

           0       0.73      0.06      0.11      1243
           1       0.51      0.98      0.67      1257

    accuracy                           0.52      2500
   macro avg       0.62      0.52      0.39      2500
weighted avg       0.62      0.52      0.39      2500



{'model': <catboost.core.CatBoostClassifier at 0x7b1f4da06d80>,
 'accuracy': 0.5208,
 'f1_weighted': 0.39091065329092245,
 'report': '              precision    recall  f1-score   support\n\n           0       0.73      0.06      0.11      1243\n           1       0.51      0.98      0.67      1257\n\n    accuracy                           0.52      2500\n   macro avg       0.62      0.52      0.39      2500\nweighted avg       0.62      0.52      0.39      2500\n',
 'cv_scores': array([0.79961561, 0.77701411, 0.7538266 , 0.79573781, 0.8193909 ])}

In [20]:
train(embeddings_method="one_hot")

[one_hot] Accuracy: 0.5012
[one_hot] F1 (weighted): 0.3440
[one_hot] CV F1 (weighted) mean±std over 5 folds: 0.7888 ± 0.0202
[one_hot] Classification report:
              precision    recall  f1-score   support

           0       0.43      0.01      0.02      1243
           1       0.50      0.99      0.67      1257

    accuracy                           0.50      2500
   macro avg       0.47      0.50      0.34      2500
weighted avg       0.47      0.50      0.34      2500



{'model': <catboost.core.CatBoostClassifier at 0x7b1f3c3828d0>,
 'accuracy': 0.5012,
 'f1_weighted': 0.344049172676192,
 'report': '              precision    recall  f1-score   support\n\n           0       0.43      0.01      0.02      1243\n           1       0.50      0.99      0.67      1257\n\n    accuracy                           0.50      2500\n   macro avg       0.47      0.50      0.34      2500\nweighted avg       0.47      0.50      0.34      2500\n',
 'cv_scores': array([0.78894898, 0.80141685, 0.75837681, 0.77767625, 0.81751438])}

In [22]:
train(embeddings_method="fasttext")

[fasttext] Accuracy: 0.6012
[fasttext] F1 (weighted): 0.6011
[fasttext] CV F1 (weighted) mean±std over 5 folds: 0.6411 ± 0.0298
[fasttext] Classification report:
              precision    recall  f1-score   support

           0       0.60      0.62      0.61      1243
           1       0.61      0.58      0.60      1257

    accuracy                           0.60      2500
   macro avg       0.60      0.60      0.60      2500
weighted avg       0.60      0.60      0.60      2500



{'model': <catboost.core.CatBoostClassifier at 0x7b1f3154fbf0>,
 'accuracy': 0.6012,
 'f1_weighted': 0.6010905478273743,
 'report': '              precision    recall  f1-score   support\n\n           0       0.60      0.62      0.61      1243\n           1       0.61      0.58      0.60      1257\n\n    accuracy                           0.60      2500\n   macro avg       0.60      0.60      0.60      2500\nweighted avg       0.60      0.60      0.60      2500\n',
 'cv_scores': array([0.60202071, 0.69128945, 0.64194987, 0.62212853, 0.64793241])}

In [23]:
train(embeddings_method="bert")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

[bert] Accuracy: 0.7980
[bert] F1 (weighted): 0.7980
[bert] CV F1 (weighted) mean±std over 5 folds: 0.8032 ± 0.0079
[bert] Classification report:
              precision    recall  f1-score   support

           0       0.80      0.79      0.79      1243
           1       0.79      0.81      0.80      1257

    accuracy                           0.80      2500
   macro avg       0.80      0.80      0.80      2500
weighted avg       0.80      0.80      0.80      2500



{'model': <catboost.core.CatBoostClassifier at 0x7b1f296ebce0>,
 'accuracy': 0.798,
 'f1_weighted': 0.7979596850332202,
 'report': '              precision    recall  f1-score   support\n\n           0       0.80      0.79      0.79      1243\n           1       0.79      0.81      0.80      1257\n\n    accuracy                           0.80      2500\n   macro avg       0.80      0.80      0.80      2500\nweighted avg       0.80      0.80      0.80      2500\n',
 'cv_scores': array([0.79392161, 0.8079754 , 0.79797171, 0.8000576 , 0.81608836])}

In [24]:
train(embeddings_method="tfidf")

[tfidf] Accuracy: 0.5176
[tfidf] F1 (weighted): 0.3833
[tfidf] CV F1 (weighted) mean±std over 5 folds: 0.7891 ± 0.0222
[tfidf] Classification report:
              precision    recall  f1-score   support

           0       0.72      0.05      0.09      1243
           1       0.51      0.98      0.67      1257

    accuracy                           0.52      2500
   macro avg       0.61      0.51      0.38      2500
weighted avg       0.61      0.52      0.38      2500



{'model': <catboost.core.CatBoostClassifier at 0x7b1f296c49b0>,
 'accuracy': 0.5176,
 'f1_weighted': 0.3833412119064493,
 'report': '              precision    recall  f1-score   support\n\n           0       0.72      0.05      0.09      1243\n           1       0.51      0.98      0.67      1257\n\n    accuracy                           0.52      2500\n   macro avg       0.61      0.51      0.38      2500\nweighted avg       0.61      0.52      0.38      2500\n',
 'cv_scores': array([0.79961561, 0.77701411, 0.7538266 , 0.79573781, 0.8193909 ])}

In [ ]:
train(embeddings_method="ppmi")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]